<a href="https://colab.research.google.com/github/Pooret/inspiritai/blob/main/squad_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from torch import tensor
from datasets import load_dataset
from transformers import DebertaV2TokenizerFast, DebertaV2ForQuestionAnswering

# Load the fast tokenizer and model
tokenizer = DebertaV2TokenizerFast.from_pretrained('microsoft/deberta-v3-base')
model = DebertaV2ForQuestionAnswering.from_pretrained('microsoft/deberta-v3-base')

dataset = load_dataset('squad')

# Function to convert a batch of samples to tensors
def convert_to_tensors(batch):
    return {k: tensor(v) for k, v in batch.items()}

# Function to tokenize and format a single example
def prepare_train_features(examples):
    # Tokenize the questions and contexts
    tokenized_inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # Truncate only the context (second part of the input)
        max_length=512,
        stride=128,  # Overlap between chunks if context is longer than max length
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Adjust answer start and end positions
    sample_mapping = tokenized_inputs.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    tokenized_inputs["start_positions"] = []
    tokenized_inputs["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_inputs.sequence_ids(i)

        sample_index = sample_mapping[i]
        answer = examples["answers"][sample_index]
        if len(answer["answer_start"]) == 0:
            tokenized_inputs["start_positions"].append(cls_index)
            tokenized_inputs["end_positions"].append(cls_index)
        else:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_inputs["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_inputs["end_positions"].append(token_end_index + 1)
            else:
                tokenized_inputs["start_positions"].append(cls_index)
                tokenized_inputs["end_positions"].append(cls_index)

    return tokenized_inputs

# Apply the function to the dataset
tokenized_train = dataset['train'].map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)
tokenized_val = dataset['validation'].map(prepare_train_features, batched=True, remove_columns=dataset["validation"].column_names)

# Convert the tokenized dataset to tensors
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'start_positions', 'end_positions'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import os
import json

best_model_path = '/content/drive/MyDrive/models/deberta_squad_weight_decay_best_model.pt'

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Convert the dataset to a DataLoader for easier batching
train_loader = DataLoader(tokenized_train, batch_size=12, shuffle=True)
val_loader = DataLoader(tokenized_val, batch_size=8, shuffle=False)


# Initial setup
num_epochs = 10 # we are using early stopping so making this large
total_steps = len(train_loader) * num_epochs
warmup_steps = total_steps * 0.1  # 10% of total steps for warmup
initial_lr = 1e-7
max_lr = 5e-5


# Use PyTorch's AdamW optimizer
optimizer = AdamW(model.parameters(), lr=initial_lr, weight_decay=0.03)

# early stopping
patience = 2
best_val_loss = float('inf')
patience_counter = 0

# learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)

for epoch in range(num_epochs):

    # training loop
    model.train()
    train_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} Training", leave=True)

    for batch_idx, batch in enumerate(progress_bar):

        current_step = epoch * len(train_loader) + batch_idx

        # Warmup phase
        if current_step < warmup_steps:
            lr_scale = min(1., float(current_step + 1) / warmup_steps)
            for pg in optimizer.param_groups:
                pg['lr'] = lr_scale * max_lr
        else:
            for pg in optimizer.param_groups:
                pg['lr'] = max_lr


        # Manually convert batch to tensors and move to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)


        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)


        # Backward pass and optimize
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Update progress bar
        progress_bar.set_postfix(avg_loss=train_loss/(batch_idx +1))

    avg_train_loss = train_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():

      for batch in val_loader:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids = input_ids,
                        attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        loss = outputs.loss
        val_loss += loss.item()

      avg_val_loss = val_loss / len(val_loader)
      scheduler.step(avg_val_loss)
      print(f"Validation Loss: {avg_val_loss:.4f}")

      # Checkpoint for the best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        print(f"Saving new best model at epoch {epoch + 1}")
        model.save_pretrained(best_model_path)
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter == patience:
        print(f"Early stopping at epoch {epoch + 1}")
        break

     # Update progress bar description to include validation loss
    progress_bar.set_description(f"Epoch {epoch + 1}/{num_epochs} Training (Val Loss: {avg_val_loss:.4f})")
    progress_bar.set_postfix(avg_train_loss=avg_train_loss, avg_val_loss=avg_val_loss)

    # Print epoch summary
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

    # If not the last epoch, update progress bar for the next epoch
    if epoch < num_epochs - 1:
        progress_bar.set_description(f"Epoch {epoch + 2}/{num_epochs} Training (Prev Val Loss: {avg_val_loss:.4f})")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/models/fine_tuned_deberta_squad_weight_decay")


Epoch 1/10 Training: 100%|██████████| 7309/7309 [1:00:47<00:00,  2.00it/s, avg_loss=1.25]


Validation Loss: 0.8436
Saving new best model at epoch 1
Epoch 1/10, Training Loss: 1.2456, Validation Loss: 0.8436


Epoch 2/10 Training: 100%|██████████| 7309/7309 [1:00:45<00:00,  2.00it/s, avg_loss=0.753]


Validation Loss: 0.8265
Saving new best model at epoch 2
Epoch 2/10, Training Loss: 0.7529, Validation Loss: 0.8265


Epoch 3/10 Training:  21%|██▏       | 1566/7309 [13:01<47:43,  2.01it/s, avg_loss=0.55]